In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from tqdm import tqdm
from Base.DataIO import DataIO

#----Recommenders----
from SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from SLIM.SlimElasticNet import SLIMElasticNetRecommender
from cf.item_cf3 import ItemBasedCollaborativeFiltering
from cf.user_cf2 import UserBasedCollaborativeFiltering
from MF.IALSRecommender import IALSRecommender
from cbf.cbf import ContentBasedFiltering
from SlimBPR.SlimBPRRec import SlimBPRRec
from SlimBPR.SlimBPR import SlimBPR
from MF.ALS import AlternatingLeastSquare


from Hybrid.hybridRec import HybridRecommender
#---------------

#----Model Load & Save----
from Data_manager.Dataset import Dataset
#-------------------------

from sklearn.model_selection import train_test_split

In [2]:
def load_data_ICM():
  return pd.read_csv("./data_ICM_title_abstract.csv")

In [3]:
features=load_data_ICM()
d ={'item_id': features['row'],'feature_id':features['col'],'value':features['data']}
features=pd.DataFrame(data=d)
itemList=list(d['item_id'])

In [4]:
featureList=list(d['feature_id'])

In [5]:
valueList=list(d['value'])
ICM = sp.coo_matrix((valueList,(itemList,featureList)))
ICM = ICM.tocsr()

In [6]:
ICM

<25975x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 490691 stored elements in Compressed Sparse Row format>

**Dataset splitting into train,validation and test**

This is the last part before creating the recommender. However, this step is super important, as it is the base for the training, parameters optimization, and evaluation of the recommender(s).

In here we read the ratings (which we loaded and preprocessed before) and create the train, validation, and test User-Rating Matrices (URM). It's important that these are disjoint to avoid information leakage from the train into the validation/test set, in our case, we are safe to use the train_test_split function from scikit-learn as the dataset only contains one datapoint for every (user,item) pair. On another topic, we first create the test set and then we create the validation by splitting again the train set.

train_test_split takes an array (or several arrays) and divides it into train and test according to a given size (in our case testing_percentage and validation_percentage, which need to be a float between 0 and 1).

After we have our different splits, we create the sparse URMs by using the csr_matrix function from scipy.




In [7]:
#Load
loaded_dataset = Dataset(None, None, None, None)
loaded_dataset.load_data("Salvataggi/")


urm_train = (loaded_dataset.get_urm_train())["urm_train"]
urm_validation = (loaded_dataset.get_urm_validation())["urm_validation"]
urm_test = loaded_dataset.get_urm_test()["urm_test"]

In [8]:
urm_train_validation = urm_train + urm_validation

**Evaluation Metrics**

In [9]:
def recall(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant) / relevant_items.shape[0]
    
    return recall_score
    
    
def precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant) / recommendations.shape[0]

    return precision_score

def mean_average_precision(recommendations: np.array, relevant_items: np.array) -> float:
    is_relevant = np.in1d(recommendations, relevant_items, assume_unique=True)
    
    precision_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(precision_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

**Evaluation Procedure**

The evaluation procedure returns the averaged accuracy scores (in terms of precision, recall and MAP) for all users (that have at least 1 rating in the test set). It also calculates the number of evaluated and skipped users. It receives a recommender instance, and the train and test URMs.

In [10]:
def evaluator(recommender: object, urm_train: sp.csr_matrix, urm_test: sp.csr_matrix):
    recommendation_length = 10
    accum_precision = 0
    accum_recall = 0
    accum_map = 0
    
    num_users = urm_train.shape[0]
    
    num_users_evaluated = 0
    num_users_skipped = 0
    
    for user_id in range(num_users):
        user_profile_start = urm_test.indptr[user_id]
        user_profile_end = urm_test.indptr[user_id+1]

        relevant_items = urm_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
            
#         recommendations = recommender.recommend(user_id_array=user_id,
#                                                cutoff=recommendation_length,
#                                                remove_seen_flag=True
#                                                )

        expected_ratings = recommender.get_expected_ratings(user_id)
        recommended_items = np.flip(np.argsort(expected_ratings), 0)

        unseen_items_mask = np.in1d(recommended_items,urm_train[user_id].indices,
                                        assume_unique=True, invert=True)

        recommendations = recommended_items[unseen_items_mask]


        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1

    
    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)
    
    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

**Tuning CBF**

In [11]:
recommenderToTune = ContentBasedFiltering(urm_train, ICM)

In [13]:
import time

best_knn = np.zeros(4000)
best_shrink = np.zeros(4000)
best_MAP = np.zeros(4000)

counter = 0
start = time.time()
for knn in range(370, 420) :

    for shrink in range(0, 40, 5) :
        
        print("Iteration " + str(counter) + " knn: " + str(knn)   + " Shrink: " + str(shrink)  + " started.")

        recommenderToTune.fit(knn, shrink, "cosine")

        accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                                urm_train_validation, 
                                                                                                urm_test)
        
        print("MAP iteration " + str(counter) + ": " + str(accum_map))
        best_knn[counter] = knn
        best_shrink[counter] = shrink
        best_MAP[counter] = accum_map
        
        counter += 1
        
stop = time.time()
print("Execution Time: " + str(stop-start))

Iteration 0 knn: 370 Shrink: 0 started.
Similarity column 25975 ( 100 % ), 1044.72 column/sec, elapsed time 0.41 min
MAP iteration 0: 0.026667813181560193
Iteration 1 knn: 370 Shrink: 5 started.
Similarity column 25975 ( 100 % ), 1140.34 column/sec, elapsed time 0.38 min
MAP iteration 1: 0.028592194708421517
Iteration 2 knn: 370 Shrink: 10 started.
Similarity column 25975 ( 100 % ), 1118.91 column/sec, elapsed time 0.39 min
MAP iteration 2: 0.02834331475283608
Iteration 3 knn: 370 Shrink: 15 started.
Similarity column 25975 ( 100 % ), 1089.19 column/sec, elapsed time 0.40 min
MAP iteration 3: 0.028219689900574826
Iteration 4 knn: 370 Shrink: 20 started.
Similarity column 25975 ( 100 % ), 1153.04 column/sec, elapsed time 0.38 min
MAP iteration 4: 0.02796455919360324
Iteration 5 knn: 370 Shrink: 25 started.
Similarity column 25975 ( 100 % ), 1040.76 column/sec, elapsed time 0.42 min
MAP iteration 5: 0.027746326511358093
Iteration 6 knn: 370 Shrink: 30 started.
Similarity column 25975 ( 1

MAP iteration 50: 0.028397939764496782
Iteration 51 knn: 376 Shrink: 15 started.
Similarity column 25975 ( 100 % ), 941.22 column/sec, elapsed time 0.46 min
MAP iteration 51: 0.028205566422650586
Iteration 52 knn: 376 Shrink: 20 started.
Similarity column 25975 ( 100 % ), 904.22 column/sec, elapsed time 0.48 min
MAP iteration 52: 0.028007092717979437
Iteration 53 knn: 376 Shrink: 25 started.
Similarity column 25975 ( 100 % ), 867.24 column/sec, elapsed time 0.50 min
MAP iteration 53: 0.027775690375012277
Iteration 54 knn: 376 Shrink: 30 started.
Similarity column 25975 ( 100 % ), 897.67 column/sec, elapsed time 0.48 min
MAP iteration 54: 0.027640506604825213
Iteration 55 knn: 376 Shrink: 35 started.
Similarity column 25800 ( 99 % ), 859.16 column/sec, elapsed time 0.50 min
Similarity column 25975 ( 100 % ), 859.94 column/sec, elapsed time 0.50 min
MAP iteration 55: 0.02760104910115241
Iteration 56 knn: 377 Shrink: 0 started.
Similarity column 25975 ( 100 % ), 891.70 column/sec, elapsed

MAP iteration 97: 0.02870037533485618
Iteration 98 knn: 382 Shrink: 10 started.
Similarity column 25975 ( 100 % ), 929.41 column/sec, elapsed time 0.47 min
MAP iteration 98: 0.02840070484598612
Iteration 99 knn: 382 Shrink: 15 started.
Similarity column 25975 ( 100 % ), 1128.58 column/sec, elapsed time 0.38 min
MAP iteration 99: 0.028219707034220512
Iteration 100 knn: 382 Shrink: 20 started.
Similarity column 25975 ( 100 % ), 983.13 column/sec, elapsed time 0.44 min
MAP iteration 100: 0.02798968698474003
Iteration 101 knn: 382 Shrink: 25 started.
Similarity column 25975 ( 100 % ), 1020.52 column/sec, elapsed time 0.42 min
MAP iteration 101: 0.02779449517833916
Iteration 102 knn: 382 Shrink: 30 started.
Similarity column 25975 ( 100 % ), 897.20 column/sec, elapsed time 0.48 min
MAP iteration 102: 0.027654195265966196
Iteration 103 knn: 382 Shrink: 35 started.
Similarity column 25975 ( 100 % ), 917.67 column/sec, elapsed time 0.47 min
MAP iteration 103: 0.027628734912976818
Iteration 104

Similarity column 25975 ( 100 % ), 930.60 column/sec, elapsed time 0.47 min
MAP iteration 144: 0.02680709168352283
Iteration 145 knn: 388 Shrink: 5 started.
Similarity column 25975 ( 100 % ), 955.10 column/sec, elapsed time 0.45 min
MAP iteration 145: 0.028710572200229312
Iteration 146 knn: 388 Shrink: 10 started.
Similarity column 25975 ( 100 % ), 905.16 column/sec, elapsed time 0.48 min
MAP iteration 146: 0.02839647990218216
Iteration 147 knn: 388 Shrink: 15 started.
Similarity column 25975 ( 100 % ), 939.27 column/sec, elapsed time 0.46 min
MAP iteration 147: 0.02824996467074061
Iteration 148 knn: 388 Shrink: 20 started.
Similarity column 25975 ( 100 % ), 916.07 column/sec, elapsed time 0.47 min
MAP iteration 148: 0.02802935945446444
Iteration 149 knn: 388 Shrink: 25 started.
Similarity column 25975 ( 100 % ), 974.47 column/sec, elapsed time 0.44 min
MAP iteration 149: 0.027830809928161584
Iteration 150 knn: 388 Shrink: 30 started.
Similarity column 25975 ( 100 % ), 995.06 column/se

Iteration 191 knn: 393 Shrink: 35 started.
Similarity column 25975 ( 100 % ), 897.91 column/sec, elapsed time 0.48 min
MAP iteration 191: 0.027721144674445787
Iteration 192 knn: 394 Shrink: 0 started.
Similarity column 25100 ( 97 % ), 836.41 column/sec, elapsed time 0.50 min
Similarity column 25975 ( 100 % ), 840.73 column/sec, elapsed time 0.51 min
MAP iteration 192: 0.02681759643118695
Iteration 193 knn: 394 Shrink: 5 started.
Similarity column 25975 ( 100 % ), 913.20 column/sec, elapsed time 0.47 min
MAP iteration 193: 0.028731500115928135
Iteration 194 knn: 394 Shrink: 10 started.
Similarity column 25975 ( 100 % ), 893.39 column/sec, elapsed time 0.48 min
MAP iteration 194: 0.02837921756611268
Iteration 195 knn: 394 Shrink: 15 started.
Similarity column 24600 ( 95 % ), 819.23 column/sec, elapsed time 0.50 min
Similarity column 25975 ( 100 % ), 825.82 column/sec, elapsed time 0.52 min
MAP iteration 195: 0.028246257941285765
Iteration 196 knn: 394 Shrink: 20 started.
Similarity colum

Iteration 235 knn: 399 Shrink: 15 started.
Similarity column 22800 ( 88 % ), 759.69 column/sec, elapsed time 0.50 min
Similarity column 25975 ( 100 % ), 774.28 column/sec, elapsed time 0.56 min
MAP iteration 235: 0.02827603914989594
Iteration 236 knn: 399 Shrink: 20 started.
Similarity column 24000 ( 92 % ), 799.13 column/sec, elapsed time 0.50 min
Similarity column 25975 ( 100 % ), 773.66 column/sec, elapsed time 0.56 min
MAP iteration 236: 0.028110672578286724
Iteration 237 knn: 399 Shrink: 25 started.
Similarity column 25975 ( 100 % ), 933.37 column/sec, elapsed time 0.46 min
MAP iteration 237: 0.02787187932620944
Iteration 238 knn: 399 Shrink: 30 started.
Similarity column 25975 ( 100 % ), 902.40 column/sec, elapsed time 0.48 min
MAP iteration 238: 0.02773997170650962
Iteration 239 knn: 399 Shrink: 35 started.
Similarity column 23700 ( 91 % ), 789.00 column/sec, elapsed time 0.50 min
Similarity column 25975 ( 100 % ), 793.09 column/sec, elapsed time 0.55 min
MAP iteration 239: 0.02

Similarity column 25975 ( 100 % ), 922.21 column/sec, elapsed time 0.47 min
MAP iteration 279: 0.02776824299498665
Iteration 280 knn: 405 Shrink: 0 started.
Similarity column 25975 ( 100 % ), 886.49 column/sec, elapsed time 0.49 min
MAP iteration 280: 0.02681778340855355
Iteration 281 knn: 405 Shrink: 5 started.
Similarity column 23300 ( 90 % ), 775.17 column/sec, elapsed time 0.50 min
Similarity column 25975 ( 100 % ), 785.39 column/sec, elapsed time 0.55 min
MAP iteration 281: 0.02875189647910986
Iteration 282 knn: 405 Shrink: 10 started.
Similarity column 24500 ( 94 % ), 816.61 column/sec, elapsed time 0.50 min
Similarity column 25975 ( 100 % ), 824.79 column/sec, elapsed time 0.52 min
MAP iteration 282: 0.028369866044701823
Iteration 283 knn: 405 Shrink: 15 started.
Similarity column 25700 ( 99 % ), 856.54 column/sec, elapsed time 0.50 min
Similarity column 25975 ( 100 % ), 855.81 column/sec, elapsed time 0.51 min
MAP iteration 283: 0.028300493522789044
Iteration 284 knn: 405 Shrin

Similarity column 25975 ( 100 % ), 1437.06 column/sec, elapsed time 0.30 min
MAP iteration 328: 0.02687031894769899
Iteration 329 knn: 411 Shrink: 5 started.
Similarity column 25975 ( 100 % ), 1424.64 column/sec, elapsed time 0.30 min
MAP iteration 329: 0.028711814201639593
Iteration 330 knn: 411 Shrink: 10 started.
Similarity column 25975 ( 100 % ), 1254.55 column/sec, elapsed time 0.35 min
MAP iteration 330: 0.02841601161224116
Iteration 331 knn: 411 Shrink: 15 started.
Similarity column 25975 ( 100 % ), 1523.88 column/sec, elapsed time 0.28 min
MAP iteration 331: 0.02826595647656872
Iteration 332 knn: 411 Shrink: 20 started.
Similarity column 25975 ( 100 % ), 1499.30 column/sec, elapsed time 0.29 min
MAP iteration 332: 0.028107484614109876
Iteration 333 knn: 411 Shrink: 25 started.
Similarity column 25975 ( 100 % ), 1306.39 column/sec, elapsed time 0.33 min
MAP iteration 333: 0.027919608923101087
Iteration 334 knn: 411 Shrink: 30 started.
Similarity column 25975 ( 100 % ), 1318.66 c

Similarity column 25975 ( 100 % ), 1318.79 column/sec, elapsed time 0.33 min
MAP iteration 379: 0.02824377969242232
Iteration 380 knn: 417 Shrink: 20 started.
Similarity column 25975 ( 100 % ), 1320.93 column/sec, elapsed time 0.33 min
MAP iteration 380: 0.02809841616417661
Iteration 381 knn: 417 Shrink: 25 started.
Similarity column 25975 ( 100 % ), 1515.33 column/sec, elapsed time 0.29 min
MAP iteration 381: 0.027934977072493643
Iteration 382 knn: 417 Shrink: 30 started.
Similarity column 25975 ( 100 % ), 1466.69 column/sec, elapsed time 0.30 min
MAP iteration 382: 0.027779046170177978
Iteration 383 knn: 417 Shrink: 35 started.
Similarity column 25975 ( 100 % ), 1231.05 column/sec, elapsed time 0.35 min
MAP iteration 383: 0.02777135949010934
Iteration 384 knn: 418 Shrink: 0 started.
Similarity column 25975 ( 100 % ), 1499.08 column/sec, elapsed time 0.29 min
MAP iteration 384: 0.026861106627637143
Iteration 385 knn: 418 Shrink: 5 started.
Similarity column 25975 ( 100 % ), 1507.20 co

In [14]:
#Tieni solo i migliori 400
knn_top400 = np.zeros(400, dtype=int)
shrink_top400 = np.zeros(400, dtype=int)
MAP_top400 = np.zeros(400)

#Sort of best maps decrescent
MAP_sorted = np.sort(best_MAP)[::-1]

#Tengo solo i primi 400 MAP
MAP_top400 = MAP_sorted[:400]

counter = 0

#MAP_top400[0] will be the highest mapping_value 
for mapping_value in MAP_top400 :
    
    for iteration in range(0, best_MAP.shape[0]) :
        
        if best_MAP[iteration] == mapping_value :
            
            knn_top400[counter] = best_knn[iteration]
            shrink_top400[counter] = best_shrink[iteration]
            
            counter += 1
            
print(knn_top400)
print(shrink_top400)
print(MAP_top400)

[404 405 403 409 406 408 407 402 389 390 394 391 412 413 395 392 393 415
 414 416 401 419 411 417 388 410 387 418 400 396 399 383 384 382 381 398
 397 385 386 380 379 378 377 373 374 375 376 370 371 372 417 419 418 416
 414 413 415 379 411 412 410 381 380 377 389 378 382 375 384 390 408 383
 376 388 409 385 391 401 395 387 396 386 397 394 398 402 403 392 400 393
 404 399 405 407 374 406 373 370 372 371 405 406 404 403 402 407 401 412
 413 414 400 395 399 398 396 397 409 410 411 408 415 388 416 394 384 417
 419 381 393 418 385 380 387 379 390 389 383 386 378 391 392 382 370 377
 376 372 371 375 374 373 401 402 403 404 400 399 405 411 415 406 410 416
 412 414 409 396 413 417 397 398 418 419 407 408 395 394 392 393 390 391
 389 386 387 385 388 384 376 377 383 379 381 380 382 378 371 370 375 374
 372 415 416 373 414 413 418 412 417 419 411 406 405 404 403 401 402 407
 408 409 410 400 398 399 396 397 395 394 393 389 390 392 387 388 391 372
 371 383 373 384 386 385 377 382 378 379 374 381 38

In [ ]:
print("Inizio tuning preciso di +10 -10 dei top400")

iteration = 0
for iteration in range(0, knn_top400.shape[0]) :
        
    start = time.time()
    print("Iteration " + str(iteration) + " knn: " + str(knn_top400[iteration])   + " Shrink: " + str(shrink_top400[iteration]+)  + " started.")
    
    print("knn +10")
    print("knn: " + str(knn_top400[iteration] + 10) + " Shrink: " + str(shrink_top400[iteration]))
    
    recommenderToTune.fit(knn_top400[iteration]+10, shrink_top400[iteration], "cosine")

    accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)

    print("MAP iteration " + str(iteration) + ": " + str(accum_map))

    if accum_map > MAP_top400[iteration] :
        
        knn_top400[iteration] = knn_top400[iteration] + 10
        shrink_top400[iteration] = shrink_top400[iteration]
        MAP_top400[iteration] = accum_map
        
        print("updated! knn+10 and shrink is better")
        
    ###############
    
    print("shrink +10")
    print("knn: " + str(knn_top400[iteration]) + " Shrink: " + str(shrink_top400[iteration]+10))
    
    recommenderToTune.fit(knn_top400[iteration], shrink_top400[iteration]+10, "cosine")

    accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)

    print("MAP iteration " + str(iteration) + ": " + str(accum_map))

    if accum_map > MAP_top400[iteration] :
        
        knn_top400[iteration] = knn_top400[iteration]
        shrink_top400[iteration] = shrink_top400[iteration]+10
        MAP_top400[iteration] = accum_map
        
        print("updated! knn and shrink+10 is better")
        
    ###############
    
    print("knn+10 and shrink +10")
    print("knn: " + str(knn_top400[iteration] + 10) + " Shrink: " + str(shrink_top400[iteration]+10))
    
    recommenderToTune.fit(knn_top400[iteration]+10, shrink_top400[iteration]+10, "cosine")

    accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)

    print("MAP iteration " + str(iteration) + ": " + str(accum_map))

    if accum_map > MAP_top400[iteration] :
        
        knn_top400[iteration] = knn_top400[iteration] +10
        shrink_top400[iteration] = shrink_top400[iteration]+10
        MAP_top400[iteration] = accum_map
        
        print("updated! knn+10 and shrink+10 is better")
        
    #######################
        
    print("knn -10")
    
    if (knn-10) > 0 :
        
        print("knn: " + str(knn_top400[iteration] - 10) + " Shrink: " + str(shrink_top400[iteration]))
        recommenderToTune.fit(knn_top400[iteration]-10, shrink_top400[iteration], "cosine")

        accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                                urm_train_validation, 
                                                                                                urm_test)

        print("MAP iteration " + str(iteration) + ": " + str(accum_map))

        if accum_map > MAP_top400[iteration] :

            knn_top400[iteration] = knn_top400[iteration] - 10
            shrink_top400[iteration] = shrink_top400[iteration]
            MAP_top400[iteration] = accum_map

            print("updated! knn-10 and shrink is better")
            
    else :
        print("knn<0")
        
    ###############
    
    print("shrink -10")
    
    if (shrink-10) > 0 :
    
        print("knn: " + str(knn_top400[iteration]) + " Shrink: " + str(shrink_top400[iteration]-10))

        recommenderToTune.fit(knn_top400[iteration], shrink_top400[iteration]-10, "cosine")

        accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                                urm_train_validation, 
                                                                                                urm_test)

        print("MAP iteration " + str(iteration) + ": " + str(accum_map))

        if accum_map > MAP_top400[iteration] :

            knn_top400[iteration] = knn_top400[iteration]
            shrink_top400[iteration] = shrink_top400[iteration]-10
            MAP_top400[iteration] = accum_map

            print("updated! knn and shrink-10 is better")
            
    else :
        print("shrink<0")
        
    ###############
    print("knn-10 and shrink -10")
    
    if (knn-10) > 0 and (shrink-10) > 0 :
        
        print("knn: " + str(knn_top400[iteration] - 10) + " Shrink: " + str(shrink_top400[iteration]-10))

        recommenderToTune.fit(knn_top400[iteration]-10, shrink_top400[iteration]-10, "cosine")

        accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                                urm_train_validation, 
                                                                                                urm_test)

        print("MAP iteration " + str(iteration) + ": " + str(accum_map))

        if accum_map > MAP_top400[iteration] :

            knn_top400[iteration] = knn_top400[iteration]-10
            shrink_top400[iteration] = shrink_top400[iteration]-10
            MAP_top400[iteration] = accum_map

            print("updated! knn-10 and shrink-10 is better")


    stop = time.time()
    print("Execution Time: " + str(stop-start))
    
    

In [ ]:
#Prendo i 5 migliori
knn_top5 = np.zeros(400)
shrink_top5 = np.zeros(400)
MAP_top5 = np.zeros(5)

#Sort of best maps decrescent
MAP_sorted = np.sort(best_MAP)[::-1]

#Tengo solo i primi 400 MAP
MAP_top5 = MAP_sorted[:5]

counter = 0

#MAP_top400[0] will be the highest mapping_value 
for mapping_value in range(0, MAP_top5) :
    
    for iteration in range(0, MAP_top400) :
        
        if best_MAP[iteration] == mapping_value :
            
            knn_top5[counter] = knn_top400[iteration]
            shrink_top5[counter] = shrink_top400[iteration]
            
            counter += 1
            
print(knn_top5)
print(shrink_top5)
print(MAP_top5)

In [ ]:
#in un intorno di 8 prendo il migliore per ognuno

for iteration in range (0, MAP_top5.shape[0]) :
    
    for knn in range ((knn_top5[iteration]-4), (knn_top5[iteration]+4)) :
        
        for shrink in range ((shrink_top5[iteration]-4), (shrink_top5[iteration]+4)) :
            
            print("knn: " + str(knn) + " Shrink: " + str(shrink)

            recommenderToTune.fit(knn, shrink, "cosine")

            accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommenderToTune, 
                                                                                                    urm_train_validation, 
                                                                                                    urm_test)

            print("MAP " + str(accum_map))

            if accum_map > MAP_top5[iteration] :

                knn_top5[iteration] = knn
                shrink_top5[iteration] = shrink
                MAP_top5[iteration] = accum_map

                print("updated! is better")
            
print(knn_top5)
print(shrink_top5)
print(MAP_top5)

In [ ]:
#best values
bestKnn = 0
bestShrink = 0
bestMap = 0

#Sort of best maps decrescent
MAP_sorted = np.sort(best_MAP)[::-1]

#Tengo solo i primi 400 MAP
bestMap = MAP_sorted[0]

counter = 0

for iteration in range(0, MAP_top5) :

    if MAP_top5[iteration] == bestMap :

        bestKnn = knn_top5[iteration]
        bestShrink = shrink_top5[iteration]

            
print(bestKnn)
print(bestShrink)
print(bestMap)

**Recommenders Parameters**

In [ ]:
#Params for the recommenders
cbf_param = {
    "knn": 380,
    "shrink": 9
}

user_cf_param = {
    "knn": 249,
    "shrink": 853
}

item_cf_param = {
    "knn": 119,
    "shrink": 430
}

slim_bpr_param = {   #work in progres....
    "learning_rate" : 0.00214168231523243,
    "epochs": 10,
    "nnz" : 0.39651777555716483,
    "knn": 1134
}

als_param = {   #work in progress...
    "n_factors": 242,
    "regularization": 2.0,
    "iterations": 15
}

Ials_param = {
    "num_factors" : 840,
    "confidence_scaling" : "linear",
    "alpha" : 0.40,
    "epsilon" : 0.28,
    "reg" : 0.0003
}


In [ ]:
userCF = UserBasedCollaborativeFiltering(urm_train)

In [ ]:
userCF.fit(knn=user_cf_param["knn"], shrink=user_cf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(userCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
itemCF = ItemBasedCollaborativeFiltering(urm_train)

In [ ]:
itemCF.fit(knn=item_cf_param["knn"], shrink=item_cf_param["shrink"], similarity="cosine")

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
SlimBpr=SlimBPRRec(urm_train)

In [ ]:
SlimBpr.fit(nnz=slim_bpr_param["nnz"], knn=slim_bpr_param["knn"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(SlimBpr, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
slim_elastic = SLIMElasticNetRecommender(urm_train)

In [ ]:
slim_elastic.fit()

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(slim_elastic, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
cbf = ContentBasedFiltering(urm_train,ICM)

In [ ]:
cbf.fit(knn=cbf_param["knn"],shrink=cbf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(cbf, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
ALS = AlternatingLeastSquare(urm_train)

In [ ]:
ALS.fit(n_factors=als_param["n_factors"], regularization=als_param["regularization"],iterations=als_param["iterations"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(ALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
IALS = IALSRecommender(urm_train)

In [ ]:
IALS.fit(num_factors=Ials_param["num_factors"], confidence_scaling=Ials_param["confidence_scaling"], alpha=Ials_param["alpha"], epsilon=Ials_param["epsilon"], reg=Ials_param["reg"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(IALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

**Hybrid**

**Hybrid Recommender Tuning**

In [ ]:
best_weight = 0
best_MAP = 0.0
counter = 0

In [ ]:
import time

#Create recommender
rec = HybridRecommender(urm_train_validation, ICM, userCF, itemCF, cbf, ALS, SlimBpr)


for w_bpr in range(0,1000) :

    start = time.time()
    print("Iteration " + str(counter) + " Weight: " + str(w_bpr)  + " started.")
    
    rec.fit(user_cf_param=user_cf_param,item_cf_param=item_cf_param,cbf_param=cbf_param,
            slim_param=slim_bpr_param,als_param=als_param, w_user=37, w_item=235, w_cbf=98, w_als=73, w_slim_bpr=w_bpr)

    accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(rec, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)

    if accum_map > best_MAP :
        best_weight = w_bpr
        print("New Best! weight: " + str(w_bpr) + " MAP: " + str(accum_map) + " previous best MAP: " + str(best_MAP))
        best_MAP = accum_map

    stop = time.time()
    print("Execution Time: " + str(stop-start))

In [ ]:
w = {
    "user_cf": 37,
    "item_cf": 235,
    "cbf": 98,
    "icm_svd": 0,
    "als": ,
    "slim_bpr": 113,
    "elastic": 0
}

In [ ]:
userCF = UserBasedCollaborativeFiltering(urm_train_validation)

In [ ]:
userCF.fit(knn=user_cf_param["knn"], shrink=user_cf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(userCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
itemCF = ItemBasedCollaborativeFiltering(urm_train_validation)

In [ ]:
itemCF.fit(knn=item_cf_param["knn"], shrink=item_cf_param["shrink"], similarity="cosine")

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(itemCF, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
SlimBpr=SlimBPRRec(urm_train_validation)

In [ ]:
SlimBpr.fit(nnz=slim_bpr_param["nnz"], knn=slim_bpr_param["knn"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(SlimBpr, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
slim_elastic = SLIMElasticNetRecommender(urm_train)

In [ ]:
slim_elastic.fit()

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(slim_elastic, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
cbf = ContentBasedFiltering(urm_train_validation,ICM)

In [ ]:
cbf.fit(knn=cbf_param["knn"],shrink=cbf_param["shrink"],similarity='cosine')

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(cbf, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
ALS = AlternatingLeastSquare(urm_train_validation)

In [ ]:
ALS.fit(n_factors=als_param["n_factors"], regularization=als_param["regularization"],iterations=als_param["iterations"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(ALS, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

In [ ]:
recommender = HybridRecommender(urm_train_validation, ICM, userCF, itemCF, cbf, ALS, SlimBpr)

In [ ]:
recommender.fit(user_cf_param=user_cf_param,item_cf_param=item_cf_param,cbf_param=cbf_param,
                slim_param=slim_bpr_param,als_param=als_param, w_user=w["user_cf"], w_item=w["item_cf"], w_cbf=w["cbf"],w_als=w["als"], w_slim_bpr=w["slim_bpr"])

In [ ]:
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped = evaluator(recommender, 
                                                                                            urm_train_validation, 
                                                                                            urm_test)
accum_precision, accum_recall, accum_map, num_user_evaluated, num_users_skipped

**Submission to competition**

This step serves as a similar step that you will perform when preparing a submission to the competition. Specially after you have chosen and trained your recommender.

For this step the best suggestion is to select the most-performing configuration obtained in the hyperparameter tuning step and to train the recommender using both the train and validation set. Remember that in the competition you do not have access to the test set.

Another consideration is that, due to easier and faster calculations, we replaced the user/item identifiers with new ones in the preprocessing step. For the competition, you are required to generate recommendations using the dataset's original identifiers. Due to this, this step also reverts back the newer identifiers with the ones originally found in the dataset.

Last, this step creates a function that writes the recommendations for each user in the same file in a tabular format following this format:

csv
<user_id>,<item_id_1> <item_id_2> <item_id_3> <item_id_4> <item_id_5> <item_id_6> <item_id_7> <item_id_8> <item_id_9> <item_id_10>
Always verify the competitions' submission file model as it might vary from the one we presented here.

In [ ]:
def load_goodguys():
  return pd.read_csv("./data_target_users_test.csv")
goodguys=load_goodguys()

In [ ]:
goodguys

In [ ]:
users_to_recommend = np.random.choice(goodguys.user_id,size=goodguys.size, replace=False)
users_to_recommend

In [ ]:
def prepare_submission(users_to_recommend: np.array, urm_train: sp.csr_matrix, recommender: object):
    
    recommendation_length = 10
    submission = []
    
    for user_id in users_to_recommend :

        recommendations = recommender.recommend2(user_id, urm_train, recommendation_length)

        
        submission.append((user_id, [item_id for item_id in recommendations]))
   
    return submission

In [ ]:
submission = prepare_submission(users_to_recommend, urm_train_validation, recommender)

In [ ]:
submission

In [ ]:
import os
from datetime import datetime

csv_fname = './submission'
csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

def write_submission(submissions):
    with open(csv_fname, "w") as f:
        f.write(f"user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")


In [ ]:
write_submission(submission)